<a href="https://colab.research.google.com/github/shivangini831-sys/FitPulse-Health-Anomaly-Detection-From-Fitness-Devices/blob/main/Copy_of_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile modeling.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer

# Load datasets
hr = pd.read_csv("data/heart_rate.csv")
steps = pd.read_csv("data/steps.csv")
sleep = pd.read_csv("data/sleep.csv")

# Prophet for heart rate
df_hr = hr.rename(columns={"timestamp":"ds", "heart_rate":"y"})
model_hr = Prophet(daily_seasonality=True)
model_hr.fit(df_hr)
future_hr = model_hr.make_future_dataframe(periods=96)
forecast_hr = model_hr.predict(future_hr)
df_hr["residual"] = df_hr["y"] - forecast_hr["yhat"][:len(df_hr)]

# Prophet for steps
df_steps = steps.rename(columns={"timestamp":"ds", "steps":"y"})
model_steps = Prophet(daily_seasonality=True)
model_steps.fit(df_steps)
future_steps = model_steps.make_future_dataframe(periods=96)
forecast_steps = model_steps.predict(future_steps)
df_steps["residual"] = df_steps["y"] - forecast_steps["yhat"][:len(df_steps)]

# Prophet for sleep
df_sleep = sleep.rename(columns={"date":"ds", "sleep_hours":"y"})
model_sleep = Prophet(weekly_seasonality=True)
model_sleep.fit(df_sleep)
future_sleep = model_sleep.make_future_dataframe(periods=14)
forecast_sleep = model_sleep.predict(future_sleep)
df_sleep["residual"] = df_sleep["y"] - forecast_sleep["yhat"][:len(df_sleep)]

# Load features CSV (from feature_extraction.py)
features = pd.read_csv("data/combined_features.csv")
imputer = SimpleImputer(strategy="mean")
features_imputed = imputer.fit_transform(features)

# PCA + KMeans
pca = PCA(n_components=2)
reduced_features = pca.fit_transform(features_imputed)
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(reduced_features)

# Save clustering results
results = pd.DataFrame({
    "pca_1": reduced_features[:,0],
    "pca_2": reduced_features[:,1],
    "cluster": clusters
})
results.to_csv("data/behavioral_clusters.csv", index=False)

# Optional visualization
plt.scatter(reduced_features[:,0], reduced_features[:,1], c=clusters)
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("Behavioral Pattern Clustering")
plt.show()


Writing modeling.py
